<a href="https://colab.research.google.com/github/HieuVuong001/AQI_analysis/blob/main/ProjectCode_CMPE255.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Before you begin


1.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.


### Import Library

In [54]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import folium
from folium.plugins import HeatMap
from google.cloud import bigquery
from google.colab import auth
import plotly.express as px

### Provide your Google Cloud credentials to the runtime

In [4]:
auth.authenticate_user()
print('Authenticated')

Authenticated


## Optional: Enable data table display

Colab includes the ``google.colab.data_table`` package that can be used to display large pandas dataframes as an interactive data table.
It can be enabled with:

In [5]:
%load_ext google.colab.data_table

If you would prefer to return to the classic Pandas dataframe display, you can disable this by running:
```python
%unload_ext google.colab.data_table
```

Declare the Cloud project ID which will be used throughout this notebook

In [6]:
project_id = 'cmpe255-project-419122'

In [7]:
!gcloud config list

[component_manager]
disable_update_check = True
[core]
account = kwanqing.ong@sjsu.edu

Your active configuration is: [default]


In [8]:
#!gcloud config set compute/region us-central1
#!gcloud config set compute/zone us-central1-a
!gcloud config list project

[core]
project (unset)

Your active configuration is: [default]


In [9]:
!gcloud config set project 'cmpe255-project-419122'

Updated property [core/project].


In [10]:
!gcloud config list project

[core]
project = cmpe255-project-419122

Your active configuration is: [default]


In [11]:
# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'cmpe255-project-419122'
client = bigquery.Client(project=project_id)

#Print your current data
for dataset in client.list_datasets():
  print(dataset.dataset_id)

aqi_trend_analysis_dataset


## AQI Trend Analysis Dataset


## Data Processing

In [ ]:
# # Path to the directory where all your CSV files are stored
# csv_directory = '/content/'

# # List to hold data from each CSV
# dataframes_list = []

# # Loop through all the CSV files in the directory
# for filename in os.listdir(csv_directory):
#     if filename.endswith('.csv'):
#         # Construct the full file path
#         filepath = os.path.join(csv_directory, filename)
#         # Read the CSV file and append it to the list
#         df = pd.read_csv(filepath)
#         dataframes_list.append(df)

# # Concatenate all DataFrames in the list into one
# annual_aqi_by_county_df = pd.concat(dataframes_list, ignore_index=True)

# # Optionally, you can save the merged DataFrame to a new CSV file
# annual_aqi_by_county_df.to_csv('/content/annual_aqi_by_county.csv', index=False)

# # Display the shape of the merged DataFrame to confirm
# print(f"Merged DataFrame shape: {annual_aqi_by_county_df.shape}")

# annual_aqi_by_county_df.head()

Merged DataFrame shape: (11176, 18)


,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10
0,Alabama,Baldwin,2016,279,247,32,0,0,0,0,87,51,37,0,0,221,58,0
1,Alabama,Clay,2016,116,109,7,0,0,0,0,56,45,30,0,0,0,116,0
2,Alabama,Colbert,2016,282,258,23,1,0,0,0,115,50,38,0,0,219,63,0
3,Alabama,DeKalb,2016,348,304,43,1,0,0,0,119,54,40,0,0,321,27,0
4,Alabama,Elmore,2016,117,107,10,0,0,0,0,77,48,40,0,0,117,0,0


In [ ]:
# # Rename columns to comply with BigQuery's column naming conventions
# annual_aqi_by_county_df = annual_aqi_by_county_df.rename(columns={
#     'Days PM2.5': 'Days_PM2_5',
# })

# # Save preprocessed data to big query
# annual_aqi_by_county_df.to_gbq(
#     'cmpe255-project-419122.aqi_trend_analysis_dataset.annual_aqi_by_county_df',
#     project_id='cmpe255-project-419122',
#     if_exists='replace',
#     progress_bar=True
# )

100%|██████████| 1/1 [00:00<00:00, 8542.37it/s]


## Exploratory Data Analysis (EDA)

In [12]:
sql = """
SELECT
    *
FROM
    `cmpe255-project-419122.aqi_trend_analysis_dataset.annual_aqi_by_county_df`
ORDER BY
  Year, State, County
"""
dfquery = client.query(sql)

In [13]:
annual_aqi_by_county_df=dfquery.to_dataframe()
annual_aqi_by_county_df.head()

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days_PM2_5,Days PM10
0,Alabama,Baldwin,2013,273,235,38,0,0,0,0,87,54,36,0,0,200,73,0
1,Alabama,Clay,2013,118,100,18,0,0,0,0,65,52,32,0,0,0,118,0
2,Alabama,Colbert,2013,285,252,33,0,0,0,0,80,51,38,0,0,202,83,0
3,Alabama,DeKalb,2013,360,319,41,0,0,0,0,93,52,38,0,0,305,55,0
4,Alabama,Elmore,2013,244,229,15,0,0,0,0,87,48,36,0,0,244,0,0


In [12]:
annual_aqi_by_county_df.tail()

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days_PM2_5,Days PM10
11171,Wyoming,Sublette,2023,211,92,113,6,0,0,0,122,84,54,0,0,209,2,0
11172,Wyoming,Sweetwater,2023,212,125,78,8,0,1,0,211,87,49,0,0,170,33,9
11173,Wyoming,Teton,2023,244,203,41,0,0,0,0,74,54,46,1,0,240,3,0
11174,Wyoming,Uinta,2023,181,181,0,0,0,0,0,32,12,6,0,0,0,0,181
11175,Wyoming,Weston,2023,181,154,27,0,0,0,0,90,58,44,0,0,181,0,0


In [13]:
annual_aqi_by_county_df.shape

(11176, 18)

In [14]:
# Information about the DataFrame
annual_aqi_by_county_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11176 entries, 0 to 11175
Data columns (total 18 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   State                                11176 non-null  object
 1   County                               11176 non-null  object
 2   Year                                 11176 non-null  Int64 
 3   Days with AQI                        11176 non-null  Int64 
 4   Good Days                            11176 non-null  Int64 
 5   Moderate Days                        11176 non-null  Int64 
 6   Unhealthy for Sensitive Groups Days  11176 non-null  Int64 
 7   Unhealthy Days                       11176 non-null  Int64 
 8   Very Unhealthy Days                  11176 non-null  Int64 
 9   Hazardous Days                       11176 non-null  Int64 
 10  Max AQI                              11176 non-null  Int64 
 11  90th Percentile AQI                  1117

In [15]:
# Summary statistics
annual_aqi_by_county_df.describe()

,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days_PM2_5,Days PM10
count,11176.0,11176.0,11176.0,11176.0,11176.0,11176.0,11176.0,11176.0,11176.0,11176.0,11176.0,11176.0,11176.0,11176.0,11176.0,11176.0
mean,2017.928239,305.679581,246.808608,54.822119,3.167144,0.730315,0.096099,0.055297,120.862115,57.351646,36.396743,0.683339,4.982015,173.512885,114.524964,11.976378
std,3.150519,87.575589,80.36493,45.526172,8.587467,3.32113,1.180406,0.699749,140.471024,17.300197,10.210863,10.400905,22.52437,117.864987,109.02719,48.47553
min,2013.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,8.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0
25%,2015.0,245.0,197.0,21.0,0.0,0.0,0.0,0.0,87.0,49.0,33.0,0.0,0.0,81.75,0.0,0.0
50%,2018.0,360.0,267.0,43.0,1.0,0.0,0.0,0.0,105.0,55.0,38.0,0.0,0.0,192.0,102.0,0.0
75%,2021.0,365.0,312.0,78.0,3.0,0.0,0.0,0.0,133.0,64.0,42.0,0.0,0.0,246.0,179.0,0.0
max,2023.0,366.0,365.0,339.0,122.0,69.0,74.0,31.0,7577.0,542.0,122.0,358.0,365.0,366.0,366.0,366.0


In [16]:
# Check for duplicate rows
print(f"Duplicate rows: {annual_aqi_by_county_df.duplicated().sum()}")

# # Remove duplicate rows
# df_no_duplicates = df_sfcrime.drop_duplicates()

Duplicate rows: 0


In [17]:
# Check for missing values
missing_values = annual_aqi_by_county_df.isnull().sum()
missing_values

State                                  0
County                                 0
Year                                   0
Days with AQI                          0
Good Days                              0
Moderate Days                          0
Unhealthy for Sensitive Groups Days    0
Unhealthy Days                         0
Very Unhealthy Days                    0
Hazardous Days                         0
Max AQI                                0
90th Percentile AQI                    0
Median AQI                             0
Days CO                                0
Days NO2                               0
Days Ozone                             0
Days_PM2_5                             0
Days PM10                              0
dtype: int64

In [20]:
# Finding distinct years
distinct_years = annual_aqi_by_county_df['Year'].unique()

# Sorting the years in ascending order (since .unique() returns a numpy array, you can use np.sort())
distinct_years_sorted = np.sort(distinct_years)

# Display the sorted unique years
print(distinct_years_sorted)

[2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023]


In [ ]:
annual_aqi_by_county_df['State'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Country Of Mexico', 'Delaware',
       'District Of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virgin Islands', 'Virginia', 'Washington', 'West Virginia',
       'Wisconsin', 'Wyoming'], dtype=object)

In [ ]:
annual_aqi_by_county_df['County'].unique()

array(['Baldwin', 'Clay', 'Colbert', 'DeKalb', 'Elmore', 'Etowah',
       'Houston', 'Jefferson', 'Lawrence', 'Madison', 'Mobile',
       'Montgomery', 'Morgan', 'Russell', 'Shelby', 'Sumter', 'Talladega',
       'Tuscaloosa', 'Aleutians East ', 'Anchorage ', 'Denali ',
       'Fairbanks North Star ', 'Juneau ', 'Kenai Peninsula ',
       'Matanuska-Susitna ', 'Yukon-Koyukuk ', 'Apache', 'Cochise',
       'Coconino', 'Gila', 'La Paz', 'Maricopa', 'Mohave', 'Navajo',
       'Pima', 'Pinal', 'Santa Cruz', 'Yavapai', 'Yuma', 'Arkansas',
       'Ashley', 'Clark', 'Crittenden', 'Garland', 'Jackson', 'Newton',
       'Polk', 'Pulaski', 'Union', 'Washington', 'Alameda', 'Alpine',
       'Amador', 'Butte', 'Calaveras', 'Colusa', 'Contra Costa',
       'Del Norte', 'El Dorado', 'Fresno', 'Glenn', 'Humboldt',
       'Imperial', 'Inyo', 'Kern', 'Kings', 'Lake', 'Los Angeles',
       'Madera', 'Marin', 'Mariposa', 'Mendocino', 'Merced', 'Mono',
       'Monterey', 'Napa', 'Nevada', 'Orange', 'Place

## Visualization

### Question 1: How has the air quality index (AQI) changed over the years in specific counties or states?


In [80]:
# Filtering the dataset for a specific state (Replace 'California' with your state of interest)
selected_state = 'California'
state_specific_data = annual_aqi_by_county_df[annual_aqi_by_county_df['State'] == selected_state]

# Grouping the data by 'Year' and calculating the average of 'Median AQI'
annual_aqi_trend = state_specific_data.groupby('Year')['Median AQI'].mean().reset_index()

# Create an interactive line chart with Plotly Express
fig = px.line(annual_aqi_trend, x='Year', y='Median AQI',
              title=f'Average Median AQI Trend Over Years in {selected_state}',
              markers=True,  # Adds markers to each point
              labels={'Median AQI': 'Average Median AQI'})  # Custom label for the y-axis

# Customize the layout
fig.update_layout(xaxis_title='Year',
                  yaxis_title='Average Median AQI',
                  xaxis=dict(tickmode='linear'),  # Ensure years are treated linearly
                  plot_bgcolor='white')  # Set background color to white for readability

# Add gridlines for better readability
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Show the figure
fig.show()

In [82]:
# Filtering the dataset for a specific county (Replace 'Ottawa' with your county of interest)
selected_county = 'Ottawa'
state_specific_data = annual_aqi_by_county_df[annual_aqi_by_county_df['County'] == selected_county]

# Grouping the data by 'Year' and calculating the average of 'Median AQI'
annual_aqi_trend = state_specific_data.groupby('Year')['Median AQI'].mean().reset_index()

# Create an interactive line chart with Plotly Express
fig = px.line(annual_aqi_trend, x='Year', y='Median AQI',
              title=f'Average Median AQI Trend Over Years in {selected_county}',
              markers=True,  # Adds markers to each point
              line_shape='linear',  # Defines the line shape to be linear
              labels={'Median AQI': 'Average Median AQI'})  # Custom label for the y-axis

# Customize the layout
fig.update_layout(xaxis_title='Year',
                  yaxis_title='Average Median AQI',
                  plot_bgcolor='white',  # Set background color to white for readability
                  xaxis=dict(tickmode='linear'))  # Ensure years are treated linearly

# Add gridlines for better readability
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Show the figure
fig.show()

In [18]:
# SQL query to calculate the average Median AQI for each year for a specific state.
q1 = """
SELECT
  Year,
  AVG(`Median AQI`) AS Average_Median_AQI
FROM
  `cmpe255-project-419122.aqi_trend_analysis_dataset.annual_aqi_by_county_df`
WHERE
  State = 'California'
GROUP BY
  Year
ORDER BY
  Year;
"""

df_q1=client.query(q1).to_dataframe()
df_q1.head(100)

,Year,Average_Median_AQI
0,2013,49.425926
1,2014,48.888889
2,2015,49.425926
3,2016,46.388889
4,2017,49.018868
5,2018,48.792453
6,2019,43.830189
7,2020,48.018868
8,2021,47.754717
9,2022,46.211538


In [19]:
# SQL query to calculate the average Median AQI for each year for a specific county.
q2 = """
SELECT
  Year,
  AVG(`Median AQI`) AS Average_Median_AQI
FROM
  `cmpe255-project-419122.aqi_trend_analysis_dataset.annual_aqi_by_county_df`
WHERE
  County = 'Ottawa'
GROUP BY
  Year
ORDER BY
  Year;
"""

df_q2=client.query(q2).to_dataframe()
df_q2.head(100)

,Year,Average_Median_AQI
0,2013,40.0
1,2014,42.5
2,2015,41.0
3,2016,40.5
4,2017,41.5
5,2018,41.5
6,2019,36.5
7,2020,35.0
8,2021,49.0
9,2022,42.0


### Question 2: How have the levels of PM2.5 and ozone changed over the years?

In [79]:
# Calculate the average number of days per year that PM2.5 and ozone were the predominant pollutants across all locations
# Aggregate the data
annual_trends = annual_aqi_by_county_df.groupby('Year').agg({
    'Days_PM2_5': 'mean',
    'Days Ozone': 'mean'
}).reset_index()

annual_trends_melted = annual_trends.melt(id_vars=['Year'], value_vars=['Days_PM2_5', 'Days Ozone'], var_name='Pollutant', value_name='Average Days')

# Create an interactive line chart
fig = px.line(annual_trends_melted, x='Year', y='Average Days', color='Pollutant', markers=True,
              line_shape='linear',  # or 'spline' for smoothed lines
              title='Annual Trends for PM2.5 and Ozone Days')

# Customize the layout
fig.update_layout(xaxis_title='Year',
                  yaxis_title='Average Number of Days',
                  legend_title='Pollutant')

fig.update_traces(mode='markers+lines')  # Combine line and marker styles
fig.show()

In [23]:
# SQL query to calculate the annual average number of days that PM2.5 was the predominant pollutants.
q3 = """
SELECT
  Year,
  AVG(`Days_PM2_5`) AS Avg_Days_PM2_5
FROM
  `cmpe255-project-419122.aqi_trend_analysis_dataset.annual_aqi_by_county_df`
GROUP BY
  Year
ORDER BY
  Year;
"""

df_q3=client.query(q3).to_dataframe()
df_q3.head(100)

,Year,Avg_Days_PM2_5
0,2013,115.812261
1,2014,115.924710
2,2015,117.126679
3,2016,110.095146
4,2017,116.221574
5,2018,120.699314
6,2019,115.636275
7,2020,123.134596
8,2021,130.583832
9,2022,121.543000


In [24]:
# SQL query to calculate the annual average number of days that ozone was the predominant pollutants.
q4 = """
SELECT
  Year,
  AVG(`Days Ozone`) AS Avg_Days_Ozone
FROM
  `cmpe255-project-419122.aqi_trend_analysis_dataset.annual_aqi_by_county_df`
GROUP BY
  Year
ORDER BY
  Year;
"""

df_q4=client.query(q4).to_dataframe()
df_q4.head(100)

,Year,Avg_Days_Ozone
0,2013,168.596743
1,2014,170.947876
2,2015,170.674664
3,2016,182.003883
4,2017,183.711370
5,2018,183.717924
6,2019,186.898039
7,2020,182.717846
8,2021,177.585828
9,2022,183.912000


### Question 3: How does the air quality (based on Median AQI) vary across different states?

In [76]:
# Calculating the average Median AQI for each state
average_aqi_by_county = annual_aqi_by_county_df.groupby('State')['Median AQI'].mean().sort_values()

average_aqi_by_county_df = average_aqi_by_county.reset_index().rename(columns={'Median AQI': 'Average Median AQI'})

# Create an interactive bar chart
fig = px.bar(average_aqi_by_county_df, x='State', y='Average Median AQI',
             labels={'Average Median AQI': 'Average Median AQI', 'State': 'State'},
             color='Average Median AQI',  # Color bars by AQI value
             color_continuous_scale='Viridis',
             title='Average Median AQI Across Different States')

# Improve layout
fig.update_layout(xaxis={'categoryorder':'total ascending'}, xaxis_title='State', yaxis_title='Average Median AQI')
fig.update_xaxes(tickangle=45)

fig.show()

In [42]:
# SQL query to calculate the average Median AQI for each state and sort the results
q5 = """
SELECT
  State,
  AVG(`Median AQI`) AS Average_Median_AQI
FROM
  `cmpe255-project-419122.aqi_trend_analysis_dataset.annual_aqi_by_county_df`
GROUP BY
  State
ORDER BY
  Average_Median_AQI;
"""

df_q5=client.query(q5).to_dataframe()
df_q5.head(100)

,State,Average_Median_AQI
0,Alaska,20.517647
1,Puerto Rico,20.627273
2,Virgin Islands,23.956522
3,Hawaii,25.325581
4,Washington,25.525836
5,Idaho,26.432161
6,Nebraska,26.694444
7,Oregon,27.765873
8,Montana,28.148325
9,Vermont,29.953488


### Question 4: What is the distribution of AQI categories across different years?

In [56]:
annual_aqi_by_county_df['Year'] = annual_aqi_by_county_df['Year'].astype(str)

# Aggregating data to get total days in each AQI category for each year
aqi_categories = ['Good Days', 'Moderate Days', 'Unhealthy for Sensitive Groups Days', 'Unhealthy Days', 'Very Unhealthy Days', 'Hazardous Days']
total_days_by_category_and_year = annual_aqi_by_county_df.groupby('Year')[aqi_categories].sum()

# Transpose the DataFrame for easier plotting
total_days_transposed = total_days_by_category_and_year.T

# Reset index to turn the AQI categories into a column.
df_long = total_days_transposed.reset_index().melt(id_vars='index', value_vars=total_days_transposed.columns, var_name='Year', value_name='Total Days')

# Rename 'index' column to 'AQI Category' for clarity
df_long.rename(columns={'index': 'AQI Category'}, inplace=True)

# Create an interactive bar chart with Plotly Express
fig = px.bar(df_long, x='AQI Category', y='Total Days', color='Year', barmode='group', title='Comparison of AQI Category Distribution Across Different Years')

# Update layout for better readability
fig.update_layout(xaxis_title='AQI Category', yaxis_title='Total Days', xaxis={'categoryorder':'total descending'}, legend_title='Year')
fig.update_xaxes(tickangle=45)
fig.show()

In [44]:
# SQL query to aggregate the total days in each Air Quality Index (AQI) category for each year across all counties and then compare these totals across different years
q6 = """
SELECT
  Year,
  SUM(`Good Days`) AS Total_Good_Days,
  SUM(`Moderate Days`) AS Total_Moderate_Days,
  SUM(`Unhealthy for Sensitive Groups Days`) AS Total_Unhealthy_for_Sensitive_Groups_Days,
  SUM(`Unhealthy Days`) AS Total_Unhealthy_Days,
  SUM(`Very Unhealthy Days`) AS Total_Very_Unhealthy_Days,
  SUM(`Hazardous Days`) AS Total_Hazardous_Days
FROM
  `cmpe255-project-419122.aqi_trend_analysis_dataset.annual_aqi_by_county_df`
GROUP BY
  Year
ORDER BY
  Year;
"""

df_q6=client.query(q6).to_dataframe()
df_q6.head(100)

,Year,Total_Good_Days,Total_Moderate_Days,Total_Unhealthy_for_Sensitive_Groups_Days,Total_Unhealthy_Days,Total_Very_Unhealthy_Days,Total_Hazardous_Days
0,2013,246492,67359,3595,625,69,46
1,2014,250142,65236,2964,462,24,31
2,2015,254068,62429,3341,631,33,33
3,2016,263552,53514,3474,461,33,37
4,2017,263992,58135,3569,892,151,62
5,2018,261814,60519,4084,1003,93,24
6,2019,271893,51360,1798,231,20,29
7,2020,276715,43000,2756,1352,319,195
8,2021,262048,58623,3833,1152,174,50
9,2022,268216,51253,2621,563,70,63


In [55]:
# Example focusing on 'Good Days' and 'Unhealthy Days' for brevity
categories_of_interest = ['Good Days', 'Unhealthy Days']
county_aqi_days = annual_aqi_by_county_df.groupby('County')[categories_of_interest].sum()
# Convert the DataFrame to a format suitable for Plotly
df_long = county_aqi_days.reset_index().melt(id_vars=['County'], value_vars=categories_of_interest, var_name='AQI Category', value_name='Total Days')

# Create an interactive bar chart
fig = px.bar(df_long, x='Total Days', y='County', color='AQI Category', orientation='h', height=2000)  # Adjust height as needed
fig.update_layout(title_text='Good vs. Unhealthy Days Across All Counties Over the Last 10 Years ', xaxis_title='Total Days', yaxis_title='County')
fig.show()